In [61]:
# Importing Libraries
import ast
import seaborn as sns
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt  

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [62]:
df_DA_IND = df[(df['job_country'] == 'India') & (df['job_title_short'] == 'Data Analyst')].copy()
df_DA_IND

job_title_short                                          job_title  \
38        Data Analyst              Data Quality and Governance Analyst 1   
123       Data Analyst  Data Analytics work from home job/internship a...   
159       Data Analyst                                       Data Analyst   
171       Data Analyst  Data Base work from home job/internship at Mga...   
194       Data Analyst                                       Data Analyst   
...                ...                                                ...   
785517    Data Analyst             Analyst - Data Analytics/SQL (2-9 yrs)   
785548    Data Analyst                                       Data Analyst   
785628    Data Analyst                                       Data Analyst   
785691    Data Analyst                     Business Intelligence Engineer   
785715    Data Analyst  Amul Careers 2023 - Apply Online - Data Analys...   

                       job_location  \
38                            India   
123                        Anywhere   
159     Hyderabad, Telangana, India   
171                        Anywhere   
194                           India   
...                             ...   
785517     Gurugram, Haryana, India   
785548  Bengaluru, Karnataka, India   
785628     Gurugram, Haryana, India   
785691   Chennai, Tamil Nadu, India   
785715                        India   

                                                  job_via job_schedule_type  \
38                                        via Trabajo.org         Full-time   
123                                          via LinkedIn         Part-time   
159                                          via LinkedIn        Contractor   
171                                          via LinkedIn         Full-time   
194                                       via BeBee India         Full-time   
...                                                   ...               ...   
785517                                    via BeBee India         Full-time   
785548                                       via LinkedIn         Full-time   
785628                                       via LinkedIn         Full-time   
785691                                       via LinkedIn         Full-time   
785715  melalui Jobsleworld - Jobs In India - Job Vaca...   Pekerjaan tetap   

        job_work_from_home search_location     job_posted_date  \
38                   False           India 2023-07-23 13:24:57   
123                   True           India 2023-03-07 13:37:56   
159                  False           India 2023-11-28 13:12:07   
171                   True           India 2023-01-14 13:11:58   
194                  False           India 2023-06-20 13:11:10   
...                    ...             ...                 ...   
785517               False           India 2023-10-16 06:10:56   
785548               False           India 2023-01-19 06:14:17   
785628               False           India 2023-01-09 06:08:12   
785691               False           India 2023-01-23 06:12:09   
785715               False           India 2023-03-13 06:16:28   

        job_no_degree_mention  job_health_insurance job_country salary_rate  \
38                      False                 False       India        None   
123                     False                 False       India        None   
159                     False                 False       India        None   
171                     False                 False       India        None   
194                     False                 False       India        None   
...                       ...                   ...         ...         ...   
785517                  False                 False       India        None   
785548                  False                 False       India        None   
785628                  False                 False       India        None   
785691                  False                 False       India        None   
78

In [63]:
df_DA_IND = df_DA_IND.dropna(subset=['salary_year_avg'])

df_DA_IND_exploded = df_DA_IND.explode('job_skills')

df_DA_IND_exploded[['salary_year_avg', 'job_skills']].head()

salary_year_avg job_skills
11412          50400.0     python
11412          50400.0        sql
11412          50400.0        aws
11412          50400.0      azure
11412          50400.0      excel

In [64]:
df_DA_skills = df_DA_IND_exploded.groupby('job_skills')['salary_year_avg'].agg(['count', 'median']).sort_values(by='count', ascending=False)

df_DA_skills = df_DA_skills.rename(columns={'count': 'skill_count', 'median': 'median_salary'})

DA_job_count = len(df_DA_IND)

df_DA_skills['skill_percent'] = df_DA_skills['skill_count'] / DA_job_count * 100

skill_percent = 7

df_DA_skills_high_demand = df_DA_skills[df_DA_skills['skill_percent'] > skill_percent]

df_DA_skills_high_demand

skill_count  median_salary  skill_percent
job_skills                                           
sql                  46       96050.00      48.936170
excel                39       98500.00      41.489362
python               36       96050.00      38.297872
tableau              20      108087.50      21.276596
r                    18       79200.00      19.148936
power bi             17      111175.00      18.085106
azure                15       93600.00      15.957447
aws                  12       79200.00      12.765957
oracle               11       79200.00      11.702128
spark                11      111175.00      11.702128
word                 10       82092.75      10.638298
looker               10      111175.00      10.638298
powerpoint           10      111175.00      10.638298

In [66]:
from adjustText import adjust_text


plt.scatter(df_DA_skills_high_demand['skill_percent'], df_DA_skills_high_demand['median_salary'])
plt.xlabel('Percent of Data Analyst Jobs')
plt.ylabel('Median Salary ($USD)')  
plt.title('Most Optimal Skills for Data Analysts in India')

from matplotlib.ticker import PercentFormatter
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: f'${int(y/1000)}K'))  
ax.xaxis.set_major_formatter(PercentFormatter(decimals=0))


texts = []
for i, txt in enumerate(df_DA_skills_high_demand.index):
    texts.append(plt.text(df_DA_skills_high_demand['skill_percent'].iloc[i], df_DA_skills_high_demand['median_salary'].iloc[i], " " + txt, fontsize=9))


adjust_text(texts, arrowprops=dict(arrowstyle='->', color='gray'))

plt.tight_layout()
plt.show()

In [67]:
sns.scatterplot(
    data=df_DA_skills_high_demand,
    x='skill_percent',
    y='median_salary',
    color='royalblue',
    marker='o',
    edgecolor='black'
)

sns.despine()
sns.set_theme(style='ticks')


texts = []
for i, txt in enumerate(df_DA_skills_high_demand.index):
    texts.append(plt.text(df_DA_skills_high_demand['skill_percent'].iloc[i], df_DA_skills_high_demand['median_salary'].iloc[i], txt))


adjust_text(texts, arrowprops=dict(arrowstyle='->', color='gray'), force_text=0.5)


plt.xlabel('Percent of Data Analyst Jobs')
plt.ylabel('Median Yearly Salary')
plt.title('Most Optimal Skills for Data Analysts in India', fontsize=14)


from matplotlib.ticker import PercentFormatter
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: f'${int(y/1000)}K'))
ax.xaxis.set_major_formatter(PercentFormatter(decimals=0))

plt.grid(True, linestyle='--', alpha=0.2)


plt.tight_layout()
plt.show()